In [1]:
DataSource_path = '/Users/sepehrbe/My_Drive/DataSources/SkinCare/archive'
Directory_path = '/Users/sepehrbe/My_Drive/DataSources/SkinCare'

In [10]:
import numpy as np
import os
import torch
import shutil
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
import time
from statistics import mean
import sklearn as sk
import warnings
from sklearn.utils import resample
import pickle as pickle
import PIL as pl
import pandas as pd
from PIL.Image import Transpose
from PIL import Image
warnings.filterwarnings('ignore')

# **Import Meta Data**

In [11]:
file = 'HAM10000_metadata.csv'
path, dir, files = os.walk(DataSource_path)
meta_data_v0 = pd.read_csv(path[0] + '/' + file)

In [12]:
# Remove Duplication
meta_data_noDup = meta_data_v0.groupby('lesion_id').first().reset_index()
# Encode Labels
meta_data_noDup.dx = pd.Categorical(meta_data_noDup.dx)
meta_data_noDup['image_label'] = meta_data_noDup.dx.cat.codes
# Reduce Nr. class 5 category to 2800
class_5 = meta_data_noDup[meta_data_noDup.image_label==5]
class_5_reduced = class_5.sample(n=2000, random_state=0)
meta_data_noDup_reduced = pd.concat([meta_data_noDup[meta_data_noDup.image_label!=5],class_5_reduced],axis=0)
# Get 
train_data, test_valid_data = train_test_split(meta_data_noDup_reduced, test_size = 0.2, random_state=0,stratify=meta_data_noDup_reduced['image_label'])
test_data, valid_data = train_test_split(test_valid_data, test_size = 0.5, random_state=0,stratify=test_valid_data['image_label'])

In [19]:
meta_data_noDup.to_csv(Directory_path+'Meta_data_noDup.csv')

synthetics_ages = pd.read_csv(Directory_path+'/synthetics_ages.csv')


In [14]:
train_data['type'] = 'train'
valid_data['type'] = 'valid'
test_data['type'] = 'test'

In [15]:
test_valid_evaluation = pd.concat([valid_data,test_data],axis=0)
test_valid_evaluation = test_valid_evaluation.groupby('image_label').head(15).reset_index()

In [17]:
test_data.image_label.value_counts(normalize=True)

5    0.491400
2    0.176904
4    0.152334
1    0.078624
0    0.056511
6    0.024570
3    0.019656
Name: image_label, dtype: float64

# **Import Images**

In [23]:
img_list = []
for path, dirs, files in  os.walk(DataSource_path):
    for f in files:
        img_list.append( path +'/' + f)
    for d in dirs:
        img_list.append( path + d)
img_list = list(set([x for x in img_list if ".jpg" in x] ))

print(len(img_list))

10015


In [24]:
# Copy Images for Augmentation Directory
Augment_Path = Directory_path + '/' + 'Augmentation'
try:
    os.makedirs(Augment_Path)
except:
    None
try:
    os.makedirs(Augment_Path+'/test')
    os.makedirs(Augment_Path+'/valid')
    os.makedirs(Augment_Path+'/train')
except:
    None
labels = list(train_data.image_label.unique())

for i in labels:
    try:
        os.makedirs(Augment_Path+'/test/'+str(i))
        os.makedirs(Augment_Path+'/valid/'+str(i))
        os.makedirs(Augment_Path+'/train/'+str(i))
    except:
        None

In [25]:
data_ = train_data[['image_id','image_label']]
name = 'train'
def copy_to_AutoEncoder(data_ , name):
    id_list = list(data_['image_id'])
    id_labels = list(data_['image_label'])
    for i in range(len(id_list)):
        for j in img_list:
            if id_list[i] in j:
                shutil.copy(j, Augment_Path+'/' + name+ '/' + str(id_labels[i]) + '/' + id_list[i] + '.jpg')
                break

data_ = train_data[['image_id','image_label']]
name = 'train'
copy_to_AutoEncoder(data_ = data_ , name = name)
data_ = valid_data[['image_id','image_label']]
name = 'valid'
copy_to_AutoEncoder(data_ = data_ , name = name)
data_ = test_data[['image_id','image_label']]
name = 'test'
copy_to_AutoEncoder(data_ = data_ , name = name)

# **Data Augmentation**

In [26]:
folder_dir = Augment_Path + '/train/'

for i in [3, 6, 0, 1]:
    f = folder_dir + str(i) + "/"
    for image in os.listdir(f):
        if "ISIC" in image and "AUG_" not in image and image.replace(".jpg", ""):
            # print("Augmented image: ", image)
            im1 = Image.open(f + image)
            augmented = im1.transpose(Transpose.FLIP_LEFT_RIGHT)
            augmented = augmented.transpose(Transpose.FLIP_TOP_BOTTOM)
            augmented.save(folder_dir + "/" + str(i) + "/AUG_" + image)
            # Add metadata of Augmented image
            new_row = train_data[train_data.image_id == image.replace(".jpg", "")]
            new_row['image_id'] = 'AUG_' + new_row['image_id'].iloc[0]
            train_data = train_data.append(new_row)

In [27]:
Meta_Data_Augmented = pd.concat([train_data, valid_data, test_data], axis=0)
Meta_Data_Augmented.to_csv(Augment_Path+'/Meta_Data_Augmented.csv')

# **Import Augmented Images and rescaling to 280x210**

In [30]:
img_list = []
for path, dirs, files in  os.walk(Augment_Path):
    for f in files:
        img_list.append( path +'/' + f)
    for d in dirs:
        img_list.append( path + d)
img_list = list(set([x for x in img_list if ".jpg" in x] ))

Images_280x210 = Directory_path + '/Images_280x210'

try:
    os.makedirs(Images_280x210)
except:
    None

for img in tqdm(img_list):
    file_out = Images_280x210 +'/'+ re.findall('\w+',img)[-2:-1][0] +'.jpg'
    os.system('convert %s %s %s' % ('-resize 46.66%',img,file_out))

100%|██████████| 4647/4647 [01:33<00:00, 49.44it/s]


# **Import Rescaled Images and convert to arrays**

In [31]:
img_list = os.listdir(Images_280x210)
Augmented_280x210_dataset = pd.DataFrame(columns=['image_id','img_array'])
for img in tqdm(img_list):
    if '.jpg' in img:
        img_name = img.replace('.jpg','')
        image = pl.Image.open( Images_280x210 +'/'+ img)
        img_array = np.asarray(image)
        new_row = pd.Series({'image_id':img_name,'img_array':img_array}, name='')
        Augmented_280x210_dataset = Augmented_280x210_dataset.append(new_row)

100%|██████████| 4648/4648 [00:07<00:00, 603.39it/s]


In [39]:
Augmented_280x210_dataset = Augmented_280x210_dataset.merge(Meta_Data_Augmented, how="left",on='image_id')
test_valid_dataset = Augmented_280x210_dataset.merge(test_valid_evaluation, how="right",on='image_id')

# **Standardization / Normalization**

In [35]:
df_ = Augmented_280x210_dataset
train_data = df_[df_.type=='train']
valid_data = df_[df_.type=='valid']
test_data = df_[df_.type=='test']
training_data = pd.concat([train_data,valid_data], axis=0)
print('Train size: ',train_data.shape[0] ,' Valid Size: ',valid_data.shape[0], ' Test size: ', test_data.shape[0])

Train size:  3833  Valid Size:  407  Test size:  407


In [34]:
training_arrays_scaled = np.stack(training_data['img_array'].values)  / 255
Mean = training_arrays_scaled.mean(axis = (0,1,2)) 
STD = training_arrays_scaled.std(axis = (0,1,2))
print(f"Mean : {Mean}   STD: {STD}") 

Mean : [0.76570269 0.55224556 0.57657015]   STD: [0.13616887 0.1481389  0.16343116]


In [41]:
#Train data standardization
train_arrays_std = np.stack(train_data['img_array'].values) / 255
for i in range(0,train_arrays_std.shape[0]):
    train_arrays_std[i] = (train_arrays_std[i] - Mean) / STD
#Validation data standardization
valid_arrays_std = np.stack(valid_data['img_array'].values) / 255
for i in range(0,valid_arrays_std.shape[0]):
    valid_arrays_std[i] = (valid_arrays_std[i] - Mean) / STD
#Test data standardization
test_arrays_std = np.stack(test_data['img_array'].values) / 255
for i in range(0,test_arrays_std.shape[0]):
    test_arrays_std[i] = (test_arrays_std[i] - Mean) / STD

#Test-Valid data standardization
test_valid_arrays_std = np.stack(test_valid_dataset['img_array'].values) / 255
for i in range(0,test_valid_arrays_std.shape[0]):
    test_valid_arrays_std[i] = (test_valid_arrays_std[i] - Mean) / STD

In [42]:
Mean_t = train_arrays_std.mean(axis = (0,1,2)) 
STD_t = train_arrays_std.std(axis = (0,1,2))
print(f"Mean : {Mean_t}   STD: {STD_t}") 

Mean : [0.00354374 0.00936217 0.00933908]   STD: [0.99341258 0.99663028 0.99648389]


In [43]:
Mean_t = test_arrays_std.mean(axis = (0,1,2)) 
STD_t = test_arrays_std.std(axis = (0,1,2))
print(f"Mean : {Mean_t}   STD: {STD_t}") 

Mean : [-0.01839491 -0.04231122 -0.04687724]   STD: [0.96394774 0.99849143 1.00558878]


In [47]:
train_arrays_std_T = torch.tensor(train_arrays_std, dtype= torch.float32).transpose(3,1)
train_labels = torch.tensor(train_data['image_label'].values.astype(np.long))

valid_arrays_std_T = torch.tensor(valid_arrays_std, dtype= torch.float32).transpose(3,1)
valid_labels = torch.tensor(valid_data['image_label'].values.astype(np.long))

test_arrays_std_T = torch.tensor(test_arrays_std, dtype= torch.float32).transpose(3,1)
test_labels = torch.tensor(test_data['image_label'].values.astype(np.long))

test_arrays_std_T = torch.tensor(test_arrays_std, dtype= torch.float32).transpose(3,1)
test_labels = torch.tensor(test_data['image_label'].values.astype(np.long))

test_valid_arrays_std_T = torch.tensor(test_valid_arrays_std, dtype= torch.float32).transpose(3,1)
test_valid_labels = torch.tensor(test_valid_evaluation['image_label'].values.astype(np.long))

input_data = (train_arrays_std_T , valid_arrays_std_T, test_arrays_std_T )
labels = (train_labels,valid_labels,test_labels)

evaluation_set = (test_valid_arrays_std_T,test_valid_labels)

In [50]:
dataset_training = Directory_path + '/DataSet_280x210'

try:
    os.makedirs(dataset_training)
except:
    None

pickle.dump(input_data, open(dataset_training + '/InputData_280x210', 'wb'))
pickle.dump(labels, open(dataset_training + '/Labels_280x210', 'wb'))
pickle.dump(evaluation_set, open(dataset_training + '/Evaluation_set', 'wb'))

In [49]:
evaluation_set[0].shape

torch.Size([105, 3, 280, 210])

In [20]:
280*210

58800

In [21]:
207*282

58374